In [1]:
!pip install kaggle
!mkdir ~/.kaggle
!echo '{"username":"beastgeetesh","key":"d18522b72211e47a429fde103d117a6b"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [2]:
import kagglehub
import os

In [3]:
import kagglehub
path = kagglehub.dataset_download("prajitdatta/movielens-100k-dataset")
print("Path to dataset files:", path)

100%|██████████| 4.77M/4.77M [00:01<00:00, 3.68MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/prajitdatta/movielens-100k-dataset/versions/1


In [8]:
# List files in the downloaded directory
print("Files in dataset directory:", os.listdir(path))

Files in dataset directory: ['ml-100k']


In [6]:
import pandas as pd

In [9]:
# Load ratings and movies data
# Replace 'u.data' and 'u.item' with the exact file names observed in the output above
ratings = pd.read_csv(os.path.join(path, "/content/u.data"), sep='\t', names=['userId', 'movieId', 'rating', 'timestamp'])
movies = pd.read_csv(os.path.join(path, "/content/u.item"), sep='|', encoding='latin-1', names=['movieId', 'title'], usecols=[0, 1])

In [10]:
# Display the first few rows to confirm loading
print(ratings.head())
print(movies.head())

   userId  movieId  rating  timestamp
0     196      242       3  881250949
1     186      302       3  891717742
2      22      377       1  878887116
3     244       51       2  880606923
4     166      346       1  886397596
   movieId              title
0        1   Toy Story (1995)
1        2   GoldenEye (1995)
2        3  Four Rooms (1995)
3        4  Get Shorty (1995)
4        5     Copycat (1995)


In [11]:
# Drop unnecessary columns if any
ratings = ratings[['userId', 'movieId', 'rating']]

In [12]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 11.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357279 sha256=8b23c9d72e5ab41eb3e1f49e84b8794f87da17191cc1b44a1bbd630f53df01b3
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [13]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

In [14]:
# Define a Reader for the Surprise library
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings, reader)

In [15]:
# Split into training and testing
trainset, testset = train_test_split(data, test_size=0.25)

In [16]:
# Initialize and train the SVD model
model = SVD()
model.fit(trainset)

In [17]:
# Evaluate the model
predictions = model.test(testset)
print("RMSE:", accuracy.rmse(predictions))

RMSE: 0.9480
RMSE: 0.9480290445870658


In [18]:
def get_recommendations(user_id, model, movies, ratings, n=10):
    # Get movies the user has rated
    rated_movie_ids = ratings[ratings['userId'] == user_id]['movieId'].tolist()

    # Filter out movies the user has already rated
    unrated_movies = movies[~movies['movieId'].isin(rated_movie_ids)]

    # Predict ratings for unrated movies
    predictions = [model.predict(user_id, movie_id) for movie_id in unrated_movies['movieId']]

    # Sort by estimated rating
    predictions.sort(key=lambda x: x.est, reverse=True)

    # Top N recommendations
    top_recommendations = predictions[:n]
    recommended_movie_ids = [pred.iid for pred in top_recommendations]
    recommended_movies = movies[movies['movieId'].isin(recommended_movie_ids)]

    return recommended_movies

# Get recommendations for user_id=1
recommendations = get_recommendations(user_id=1, model=model, movies=movies, ratings=ratings)
print(recommendations)

      movieId                                   title
275       276                Leaving Las Vegas (1995)
284       285                   Secrets & Lies (1996)
356       357  One Flew Over the Cuckoo's Nest (1975)
407       408                   Close Shave, A (1995)
482       483                       Casablanca (1942)
527       528              Killing Fields, The (1984)
646       647                              Ran (1985)
655       656                                M (1931)
922       923            Raise the Red Lantern (1991)
1006     1007              Waiting for Guffman (1996)
